# Tutorial on JIT versus Scipy execution within Parcels

This very brief tutorial is meant to highlight the potentially very big difference between the computational time required to run Parcels in JIT (Just-In-Time compliation) versus in Scipy mode.

In the code snippet below, we use `AdvectionRK4` to advect 100 particles in the peninsula `FieldSet`. We first do it in JIT mode (by setting `ptype=JITParticle` in the declaration of `pset`) and then we also do it in Scipy mode (by setting `ptype=ScipyParticle` in the declaration of `pset`).

In both cases, we advect the particles for 1 hour, with a timestep of 30 seconds.

To measure the computational time, we use the `timer` module.

In [1]:
from parcels import FieldSet, ParticleSet, JITParticle, ScipyParticle
from parcels import AdvectionRK4
from parcels import timer
from datetime import timedelta as delta

timer.root = timer.Timer('root')

timer.fieldset = timer.Timer('fieldset creation', parent=timer.root)
fieldset = FieldSet.from_parcels('Peninsula_data/peninsula', allow_time_extrapolation=True)
timer.fieldset.stop()

ptype = {'scipy': ScipyParticle, 'jit': JITParticle}
ptimer = {'scipy': timer.Timer('scipy', parent=timer.root, start=False),
          'jit': timer.Timer('jit', parent=timer.root, start=False)}

for p in ['scipy', 'jit']:
    pset = ParticleSet.from_line(fieldset=fieldset, pclass=ptype[p], size=100, start=(3e3, 3e3), finish=(3e3, 45e3))

    ptimer[p].start()
    pset.execute(AdvectionRK4, runtime=delta(hours=1), dt=delta(seconds=30))
    ptimer[p].stop()

timer.root.stop()
timer.root.print_tree()

100% (3600.0 of 3600.0) |################| Elapsed Time: 0:00:00 Time:  0:00:00
INFO: Compiled JITParticleAdvectionRK4 ==> /var/folders/r2/8593q8z93kd7t4j9kbb_f7p00000gr/T/parcels-504/593c38d38f7ab12f970ce11dfe00d106.so
100% (3600.0 of 3600.0) |################| Elapsed Time: 0:00:00 Time:  0:00:00


(100%)  Timer root                       : 1.009e+01 s
(  1%)    (  1%) Timer fieldset creation : 1.138e-01 s
( 73%)    ( 73%) Timer scipy             : 7.326e+00 s
( 26%)    ( 26%) Timer jit               : 2.628e+00 s


As you can see above, even in this very small example **Scipy mode took almost 3 times as long** (7.3 seconds versus 2.6 seconds) as the JIT mode. For larger examples, this can grow to hundreds of times slower.

This is just an illustrative example, depending on the number of calls to `AdvectionRK4`, the size of the `FieldSet`, the size of the `pset`, the ratio between `dt` and `outputdt` in the `.execute` etc, the difference between JIT and Scipy can vary significantly. However, JIT will almost always be faster!

So why does Parcels support both JIT and Scipy mode then? Because Scipy is easier to debug when writing custom kernels, so can provide faster development of new features.

*As an aside, you may wonder why we use the `time.time` module, and not the timeit module, to time the runs above. That's because it affects the AST of the kernels, causing errors in JIT mode.*